**Word2Vec** é um método para gerar *Word Embeddings* a partir de um corpus de texto, utilizando redes neurais.

Desenvolvido por Tomas Mikolov *et al.* (Google) em 2013, é um dos métodos de geração de *word embeddings* mais populares em tarefas de processamento de linguagem natural (PLN) como análise de sentimento, tradução de textos e reconhecimento de entidades nomeadas (NER).


### Passo 1 - importanto as bibliotecas
Vamos primeiro instalar e importar as bibliotecas que utilizaremos.

In [1]:
import re
import numpy as np
from gensim import corpora, models, similarities
import nltk
import pickle
import pandas as pd
import unicodedata
import spacy

D:\Users\roger\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Para o pré-processamento em língua portuguesa, vamos usar o pacote ```pt_core_news_sm```. Instale antes com:
    
```python -m spacy download pt_core_news_sm```

In [2]:
nlp = spacy.load('pt_core_news_sm') 

# Passo 2 - Pré-processamento do *corpus*


Vamos abrir o arquivo e transformá-lo em um *dataframe*.

In [3]:
file = open(r"incor.txt", "r", encoding='ISO-8859-1')
df = pd.DataFrame(file)
df.columns = ['lines']
df = df.sort_index()
file.close()

Assim são os textos.

In [4]:
df

,lines
0,<IDENTIFICACAO DO PACIENTE: 10000237>\n
1,<SEQUENCIA DO EVENTO: 3>\n
2,<INTERVALO EM DIAS ENTRE OS EVENTOS: 77>\n
3,\n
4,28/08/2014\n
...,...
1040122,"Retorno na vaga, com exames.\n"
1040123,\n
1040124,R3 ######## ######\n
1040125,\n


Vamos primeiro remover os acentos;

In [5]:
def remove_accents(text):
    '''Strip accents out.'''
    return ''.join(c for c in unicodedata.normalize('NFD', text)
                   if unicodedata.category(c) != 'Mn')

clean2 = lambda x: cleaning1(x)

In [6]:
df = pd.DataFrame(df.lines.apply(remove_accents))

In [7]:
df

,lines
0,<IDENTIFICACAO DO PACIENTE: 10000237>\n
1,<SEQUENCIA DO EVENTO: 3>\n
2,<INTERVALO EM DIAS ENTRE OS EVENTOS: 77>\n
3,\n
4,28/08/2014\n
...,...
1040122,"Retorno na vaga, com exames.\n"
1040123,\n
1040124,R3 ######## ######\n
1040125,\n


Agora, vamos limpar, usando expressões regulares, tudo que não é caracter alfanumérico e passar tudo para caixa baixa.


In [8]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['lines'])

In [9]:
brief_cleaning

<generator object <genexpr> at 0x0000020EBB931820>

Vamos lematizar e retirar as *stopwords*, para diminuir a dimensionalidade, com a biblioteca ```spacy```. Vamos usar ```spacy.pipe()```.

In [10]:
def cleaning(doc):
    # lematizar e retirar stopwords
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # como Word2Vec usa palavras de contexto para aprender a representação vetorial de uma palavra-alvo,
    # se uma frase tiver apenas uma ou duas palavras,
    # o benefício para o treinamento é muito pequeno    
    if len(txt) > 2:
        return ' '.join(txt)

In [11]:
txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_process=-1)]

In [12]:
txt[681:686]

['  negar sincopar pra sincopar',
 '  negar edema mmii usar lasix mencionar restria ha drica l dia',
 '  negar perda ponderal',
 None,
 '  ef p est imc']

In [13]:
txt2 = [str(row.strip()) for row in txt if (row != None and row.strip() != '.')]

In [14]:
txt2[632:634]

['paciente encaminhar intersticio sianis sugestivo hp desproporcinal aumentar psp ano aumentar ca maras direito mov paradoxal septo alar paciente internar recentemente reumatologia infea respiratio pôr imagem tomografia ta rax sugestivo progressa doena comparar mesmo internaa iniciar sildenafil mg h',
 'atual cf iii referir melhorar internaa negar dor torar cica sincopar tontura']

Vamos remover os valores faltantes e duplicados.

In [15]:
df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean

,clean
0,identificacao paciente
1,sequencia evento
2,intervalar dia evento
6,ambulaa rir sono
11,rm ago ps diag enxertar c radial p dp cd mam...
...,...
1040115,lab hb leuco plt col hdl ldl trig cpk gli u ...
1040118,trocar clortalidona mg xdia aumentar clonidina...
1040119,solicitar mapa retornar
1040121,orientar caminhar cobrar retornar


In [16]:
len(txt2)

674083

#### Bigramas
O pacote ```Gensim Phrases``` detecta automaticamente frases comuns (bigramas) de uma lista de frases.

Veja mais em: https://radimrehurek.com/gensim/models/phrases.html

Com o método ```Phrases()```, vamos criar frases relevantes a partir da lista de frases.


In [17]:
from gensim.models.phrases import Phrases, Phraser

sent = [row.split() for row in df_clean['clean']]

In [18]:
phrases = Phrases(sent, min_count=30, progress_per=10000)

```Phraser()``` reduz o consumo de memória de ```phrases``` ao descartar estado do modelo não necessário para a detecção de bigramas.

In [19]:
bigram = Phraser(phrases)

Transformamos o corpus com base nos bigramas detectados.

In [20]:
sentences = bigram[sent]

#### Palavras frequentes
Vamos calcular a frequencia das palavras, para verificar a eficácia da lematização, remoção de palavras irrelevantes e adição de bigramas.

In [21]:
from collections import defaultdict 

word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

48183

In [22]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['mg', 'comp', 'vos', 'xd', 'x', 'dia', 'negar', 'paciente', 'referir', 'usar']

### Passo 3 - Treinamento do modelo

Hora de treinar nosso modelo Word2Vec com nossos dados. Primeiro, vamos verificar o ambiente.

In [23]:
import multiprocessing
from gensim.models import Word2Vec
from time import time

cores = multiprocessing.cpu_count() # conta o número de núcleos do computador

Vamos parametrizar nosso modelo.

- *min_count*: Ignora todas as palavras com frequência absoluta total inferior a esta
- *window*: A distância máxima entre a palavra atual e a prevista em uma frase. Por exemplo, palavras da janela à esquerda e palavras da janela à esquerda do nosso alvo
- *size*: Dimensionalidade dos vetores
- *sample*: O limite para configurar quais palavras de alta frequência são reduzidas aleatoriamente
- *alpha*: A taxa de aprendizagem inicial
- *min_alpha*: A taxa de aprendizado cairá linearmente para *min_alpha* conforme o treinamento progride
- *negative*: Se > 0, a amostragem negativa será usada, o int para negativo especifica quantas "palavras de ruído" devem ser eliminadas. Se definido como 0, nenhuma amostra negativa é usada
- *workers*: Quantidade de *threads* de trabalho para treinar o modelo (treinamento mais rápido com máquinas multicore)

In [24]:
w2v_model = Word2Vec(min_count=3,
                     window=2,
                     vector_size=32,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=10,
                     workers=cores-1)

#### Construindo o vocabulário e treinando o modelo
Parâmetros do treinamento:

*total_examples: Contagem de sentenças;
*epochs*: Número de iterações (épocas) no corpus

In [25]:
w2v_model.build_vocab(sentences, progress_per=10000)


In [26]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)


(47035010, 86169630)

### Passo 4 - Salvando o modelo
Vamos salvar o modelo nos formatos *KeyedVectors* e binário, para utilizarmos posteriormente.

In [28]:
w2v_model.init_sims(replace=True)  # deixa o modelo mais eficiente, pois não vamos mais treiná-lo futuramente

<ipython-input-28-bc56ad85d6ac>:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)  # deixa o modelo mais eficiente, pois não vamos mais treiná-lo futuramente


In [29]:
w2v_model.save("corpus_incor.model")
w2v_model.wv.save_word2vec_format('corpus_incor.bin', binary=True)



Agora que já treinamos nosso modelo, [vamos ver aqui como utilizá-lo](https://github.com/lisaterumi/word2vec-harry-potter-portugues/blob/main/%5B2%5D%20tSNE-Harry-Potter.ipynb).